In [552]:
import pandas as pd
import re
import gensim
import nlm
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [553]:
columns = ["sentiment", "id", "date", "query", "user_id", "text"]
df = pd.read_csv("data.csv", encoding="latin", names=columns)
subdf = df.iloc[0:1000]
text = []
sen = []  # sentences

In [554]:


for row in subdf["text"]:
    row = row.lower()
    row = re.sub(r'((www.\S+)|(https?://\S+))', r"", row)
    row = re.sub(r'(@\S+) | (#\S+)', r'', row)
    row = row.replace(",", " #c")  # we want to keep punctuation
    row = row.replace(".", " #p")
    row = row.replace("!", " #e")
    row = row.replace("?", " #q")
    sen.append(re.split("[\n\s:;]", row))

In [555]:
model = gensim.models.Word2Vec(
    sentences=sen, vector_size=25, window=10, epochs=100, min_count=1, workers=4
)

In [556]:
model.save("project3.model")


In [557]:
model = gensim.models.Word2Vec.load("project3.model")
vocab = list(model.wv.key_to_index.keys())
vocab_len = len(model.wv)
print(vocab)


['', '#p', 'i', '#e', 'to', 'the', '#c', 'my', 'a', 'and', 'is', '#q', 'in', 'it', 'for', 'you', 'me', 'of', 'so', "i'm", 'that', 'but', 'have', 'was', 'on', 'not', 'this', 'at', 'just', 'with', 'no', 'out', 'get', 'be', 'all', 'too', 'sad', 'now', 'like', 'up', "don't", "it's", 'work', 'time', 'sleep', 'still', 'go', 'going', 'back', 'really', 'day', 'one', 'know', '-', 'got', 'has', 'today', 'from', 'do', "can't", 'are', 'how', 'am', 'miss', 'good', 'much', 'an', 'they', 'about', 'want', 'some', 'when', 'its', 'what', 'im', 'night', 'oh', 'sorry', 'your', 'tomorrow', 'see', 'why', 'home', 'feel', 'had', 'u', 'bad', 'think', 'he', 'need', 'hate', 'only', 'if', 'new', '2', 'we', 'more', 'been', 'well', "didn't", '&quot', 'again', 'her', 'can', 'as', 'sick', 'off', 'them', 'here', 'will', 'wish', 'love', 'him', 'though', '3', 'did', 'bed', 'last', 'getting', 'she', 'lol', 'school', "i've", 'people', 'lost', 'then', 'hope', 'tonight', 'down', 'over', '&amp', 'hear', 'thing', 'find', 'bet

In [558]:
epochs = 1
lm = nlm.LanguageModel(vocab_len)  # Adjust hidden_size as needed
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(lm.parameters(), lr=0.01)


In [559]:
vocab.append('<unk>')
# Populate the 'text' variable
for row in subdf["text"]:
    row = row.lower()
    row = re.sub(r'((www.\S+)|(https?://\S+))', r"", row)
    row = re.sub(r'(@\S+) | (#\S+)', r'', row)
    row = row.replace(",", " #c")
    row = row.replace(".", " #p")
    row = row.replace("!", " #e")
    row = row.replace("?", " #q")
    text += re.split("[\n :;]", row)

In [560]:
for epoch in range(epochs):
    for i in range(3, len(text)-1):
        input = [list(model.wv[text[i - 3]]), list(model.wv[text[i - 2]]), list(model.wv[text[i - 1]])]
        input = [item for sublist in input for item in sublist]  # Flatten the list

        target_word_index = vocab.index(text[i + 1])  # Predict the next word in the sequence
        y_pred = lm(torch.tensor(input, dtype=torch.float))
        y_true = torch.tensor([target_word_index], dtype=torch.long)

        loss = loss_fn(y_pred.unsqueeze(0), y_true)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

print("epoch finished")

epoch finished


In [561]:
# Save the model state dictionary
torch.save(lm.state_dict(), "language.pth")

In [562]:
lm = nlm.LanguageModel(vocab_len)
lm.load_state_dict(torch.load("language.pth"))

<All keys matched successfully>

In [574]:
import torch.nn.functional as F
import numpy as np

def complete_sentence(prompt, language_model, word2vec_model, vocab, max_length=10):
    input_words = prompt.lower().split()

    for _ in range(max_length):
        input_vectors = [word2vec_model.wv.get_vector(word) if word in word2vec_model.wv else [0.0] * word2vec_model.vector_size for word in input_words[-3:]]
        input_vectors_flat = [item for sublist in input_vectors for item in sublist]

        try:
            input_tensor = torch.tensor(input_vectors_flat, dtype=torch.float).unsqueeze(0)
            output_probs = F.softmax(language_model(input_tensor), dim=1).squeeze().detach().numpy()
            predicted_index = np.random.choice(len(output_probs), p=output_probs)
            predicted_word = vocab[predicted_index]
        except (ValueError, KeyError):
            predicted_word = '<unk>'

        input_words.append(predicted_word)
        if predicted_word in ["#c", "#p", "#e", "#q"]:
            break  # Stop if a punctuation mark is predicted

    cleaned_sentence = ' '.join([word for word, next_word in zip(input_words, input_words[1:] + ['']) if word != '<unk>' or next_word != '<unk>'])

    return cleaned_sentence


In [575]:
prompt_sentence = "This is a"
completed_sentence = complete_sentence(prompt_sentence,lm, model,vocab)
print("Prompt: ", prompt_sentence)
print("Completed Sentence: ", completed_sentence)


Prompt:  This is a
Completed Sentence:  this is a 1110 eyebrows woman screams crummy one lazy alot partion today
